In [ ]:
import numpy as np
import pandas as pd
from PIL import Image
from collections import Counter
from matplotlib import pyplot as plt

from sklearn.svm import SVC
import sklearn.metrics.pairwise as Ker
from sklearn.metrics.pairwise import rbf_kernel

import os
import sys
import time
import copy
import pickle
import multiprocessing

In [ ]:
import DataLoader as D
from sklearn.metrics import classification_report

import cv2 as cv
import tensorflow as tf
from finetune_alexnet_with_tensorflow.alexnet import AlexNet
from finetune_alexnet_with_tensorflow.caffe_classes import class_names

In [ ]:
def get_cost_time(diff) :
    second = diff % 60
    minute = diff % 3600
    minute = minute // 60
    hour = diff % 86400
    hour = hour // 3600
    day = diff // 86400
    format_string="{}d {}h:{}m:{}s"
    return format_string.format(day, hour, minute, second)

In [ ]:
def evaluation(pre, ground, g_l, f_l, dataType=None, ori_labels=None):
    """function: evaluate the result according to the accuracy, FAR, FRR, HTER
        args:
            pre: (n,), prediction
            ground: (n,), ground truth
            g_l: int, the label of bona fide example
            f_L: int, the label of example from attack attempt
        return:
            acc: accuracy
            FAR: false accept rate
            FRR: false reject rate
            HTER: half total error rate
    """
    acc = (ground==pre).sum() / len(ground)
    
    tr_index = np.where(pre==g_l)
    fa_index = np.where(pre==f_l)
    FAR = len(np.where(ground[tr_index]==f_l)[0])/len(np.where(ground==f_l)[0])
    FRR = len(np.where(ground[fa_index]==g_l)[0])/len(np.where(ground==g_l)[0])
    HTER = (FAR+FRR)/2
    
    return acc, FAR, FRR, HTER

In [ ]:
def PCA_train(X, k):
    """function: calculating eigenvector from X 
        
        args:
            X: Data
            k: number of component
            
        return:
            eigenvector (k, origin_n_features)
    """
    start_time = time.time()
    print("start the fitting of PCA")
    
    #mean,std of each feature
    n_samples, n_features = X.shape
    mean = np.array([np.mean(X[:,i]) for i in range(n_features)])
    std = np.array([np.std(X[:,i]) for i in range(n_features)])
    #normalization
    norm_X = np.divide(X - mean, std, where=std!=0)
    #scatter matrix
    scatter_matrix = np.dot(np.transpose(norm_X), norm_X)
    #Calculate the eigenvectors and eigenvalues
    eig_val, eig_vec = np.linalg.eig(scatter_matrix)
    eig_pairs = [(np.abs(eig_val[i]), eig_vec[:,i]) for i in range(n_features)]
    # sort eig_vec based on eig_val from highest to lowest
    eig_pairs = sorted(eig_pairs, key=lambda x: x[0], reverse=True)
    # select the top k eig_vec
    feature=np.array([ele[1] for ele in eig_pairs[:k]])
    feature = feature.astype(np.float32)
    
    end_time = time.time()
    cost_time = get_cost_time(end_time-start_time)
    print("The time cost of PCA training: ", cost_time)
    
    if (feature!=feature).sum() != 0 :
        raise Exception("There are some nan or inf in PCA: {}".format((feature!=feature).sum()))
    
    return feature

In [ ]:
def PCA_transform(X, feature):
    """function: transforming X with calculated eigenvector
        
        args:
            X: data to be transformed
            feature: eigenvector
            
        return:
            data after transformed and normalization, (n_sample, k)
    """
    # normalization
    n_samples, n_features = X.shape
    mean = np.array([np.mean(X[:,i]) for i in range(n_features)])
    std = np.array([np.std(X[:,i]) for i in range(n_features)])
    norm_X = np.divide(X - mean, std, where=std!=0)
#     norm_X = (X - mean) / std
    # get new data
    data = np.dot(norm_X, np.transpose(feature))
    res = data
    # normalization
    n_samples, n_features = data.shape
    mean = np.array([np.mean(data[:,i]) for i in range(n_features)])
    std = np.array([np.std(data[:,i]) for i in range(n_features)])
    res = np.divide(data - mean, std, where=std!=0)
    
    return res

In [ ]:
def DK(data, group_index, is_traing=False, pca=None, gamma=0.00015):
    """function: extract DK features of each video's frame set
        
        args:
            data:   frames of all video, (n_frames, n_features)
            group_index:   all index of each video's frames in video_set, which is a dictionary
                        like, "real1":([0,1,2,...,8], label), "fake1":([9,10,..,32], label), ...
            
        return:
            while training:
                representation:   the DK features of each video
                labels:   labels of each video
                pca:   the eigenvector
            while testing:
                representation:   the DK features of each video
                labels:   labels of each video
    """
#     reduce the dimension
    data = np.array([ (vec-vec.mean())/vec.std() for vec in data.transpose() ]).transpose()
    if is_traing == True:
        pca = PCA_train(data, k=100)
    
    
#     extract DK feature and label of each video
    representation = []
    labels = []
    
    
    for key in group_index.keys():
        labels.append(group_index[key][1])
        frames = np.array(data[ group_index[key][0] ])
        
        if (frames != frames).sum() > 0:
            print(key, (frames != frames).sum())
        frames = PCA_transform(frames, pca)
        if (frames != frames).sum() > 0:
            print(key, (frames != frames).sum())
        frames = frames.transpose()
        
        res = rbf_kernel(frames, gamma=gamma)
#         res = res[ np.triu_indices(res.shape[0]) ]
        representation.append(res.flatten())
    
    if is_traing:
        return representation, labels, pca
    else:
        return representation, labels

In [ ]:
def Save_KM_label(X, Y, labels, ker_type, mat_txt_name=None, label_txt_name=None, gamma=0.0007):
    """function: calculate the kernel matrix, and save it as txt_name
    
        args:
            X: the first data of kernel matrix operation
            Y: the second data of kernel matrix operation
            labels: the labels of data
            ker_type: which kind of kernel to use
            mat_txt_name: the name of matrix file
            label_txt_name: the name of labels file
            gamma: the parameter of rbf kernel
        
        return:
            the kernel matrix
        
    """
    X = np.array(X)
    Y = np.array(Y)
    if X.shape[1] != Y.shape[1]:
        raise Exception("the second dimension of X and Y are different   ", X.shape, Y.shape)
    if X.shape[0] != len(labels):
        raise Exception("labels are not corresponding to X   ", X.shape, len(labels))
    
    if ker_type == "rbf":
        print("using RBF kernel with gamma=%.6f" % (gamma))
        ker_matrix = rbf_kernel(X, Y, gamma=gamma)
        
    elif ker_type == "linear":
        print("using linear kernel")
        ker_matrix = Ker.linear_kernel(X, Y)
        
    else:
        raise Exception("Kernel Type Error")
    print("Done.   ", ker_matrix.shape)
    
    if mat_txt_name != None and label_txt_name != None:
        np.savetxt(mat_txt_name, ker_matrix, fmt="%.9f")
        np.savetxt(label_txt_name, labels, fmt="%d")
        print("Saved - '%s' - '%s'" % (mat_txt_name, label_txt_name) )
    else:
        print("donot save the matrix and label")
    
    return ker_matrix

In [ ]:
def Deep(group_index, g_l=1, f_l=-1, random=True):
    """function: compute the deep feature of a frame randomly selected from the corresponding video 
        args: 
            group_index: (the inex of frames, label, the paths of frames)
        
        return:
            fea_set: the deep feature
            labels: the labels
    """
    group_keys = []
    labels = []
    paths = []
    for key in group_index.keys():
        if len(group_index[key][0]) == 0:
            print("There is no faces in ", key)
            continue
        
        # get the label
        labels.append(group_index[key][1])
        
        # randomly select a image's path
        tmp_path_set = group_index[key][2]
        if random is True :
            path = tmp_path_set[np.random.randint(0, len(tmp_path_set))]
            paths.append( path.strip() )
        else :
            for path in tmp_path_set :
                paths.append( path.strip() )
        group_keys.append(key)
    
    
    # create the graph
    tf.reset_default_graph()
    
    #mean of imagenet dataset in BGR
    imagenet_mean = np.array([104., 117., 124.], dtype=np.float32)

    with tf.device('/gpu:0'):
        #placeholder for input and dropout rate
        x = tf.placeholder(tf.float32, [1, 227, 227, 3])
        keep_prob = tf.placeholder(tf.float32)

        #create model with default config ( == no skip_layer and 1000 units in the last layer)
        model = AlexNet(x, keep_prob, 2, [])

        #define activation of last layer as score
        score = model.fc8
        fea = model.fc7

        #create op to calculate softmax 
        softmax = tf.nn.softmax(score)
    
    # saver=tf.train.import_meta_graph('./finetune_alexnet_with_tensorflow/log/checkpoints/model_epoch10.ckpt.meta')
    saver = tf.train.Saver(tf.global_variables())
    
    config = tf.ConfigProto(log_device_placement=True, allow_soft_placement=True)
    config.gpu_options.allow_growth = True
    
    fea_set = []
    pre_set = []
    with tf.Session(config=config) as sess:
        # initialize the variables
        init_op = tf.group(
                tf.global_variables_initializer(),
                tf.local_variables_initializer()
                )
        sess.run(init_op)
        
        # restore the weights of pretrained network
        saver.restore(sess, tf.train.latest_checkpoint('./finetune_alexnet_with_tensorflow/log/checkpoints-REPLAY-hsv/'))
        
        # Loop over all images
        for i, path in enumerate(paths):
            # read the image for color image
#             image = cv.imread(path)
            image = cv.imread(path, cv.IMREAD_GRAYSCALE)
            image = np.stack((image,image,image), axis=2)
            
            # Convert image to float32 and resize to (227x227)
            img = cv.resize(image.astype(np.float32), (227,227))
            
            # Subtract the ImageNet mean
            img -= imagenet_mean
            
            # Reshape as needed to feed into model
            img = img.reshape((1,227,227,3))

            # Run the session and calculate the class probability
            probs, res_fea= sess.run([softmax, fea], feed_dict={x: img, keep_prob: 1})
            
            # Get the class name of the class with the highest probability
            class_name = np.argmax(probs)
            
            # store the feature from fc7 and the predicted class
            fea_set.append(res_fea.flatten())
            pre_set.append(class_name)
            
            if np.mod((i+1), 100) == 0 :
                print("\riteration: {}".format(i+1), end="")
    
#     fea_set = np.array(fea_set)
#     pre_set = np.array(pre_set)
#     labels = np.array(labels)
#     pre_set[ np.where(pre_set==0) ] = f_l
#     acc, FAR, FRR, HTER = evaluation(pre_set, labels, g_l=g_l, f_l=f_l, dataType=None, ori_labels=None)
#     print("acc: {}, FAR: {}, FRR: {}, HTER: {}".format(acc, FAR, FRR, HTER))
    
    return fea_set, labels, group_keys

In [ ]:
def average_deepFea(deep_fea, group_index) :
    res_list = []
    for key in group_index.keys():
#         print(type(group_index[key][0]), group_index[key][0][:10])
        fea_set = np.array(deep_fea[ group_index[key][0] ])
        res_list.append(fea_set.mean(axis=0))
    
    return np.array(res_list)

In [ ]:
train_path = "./Data/CASIA_train_HSV_64_15diff.pkl"
test_path = "./Data/CASIA_test_HSV_64_15diff.pkl"
# train_path = "./Data/REPLAY_train_HSV_64_15diff.pkl"
# test_path = "./Data/REPLAY_test_HSV_64_15diff.pkl"
# train_path = "./Data/NUAA_train_GRAY_64_15diff_frame.pkl"
# test_path = "./Data/NUAA_test_GRAY_64_15diff_frame.pkl"

saver = D.Saver(train_path)
train_images_set, train_group_index = saver.load()
train_images_set = train_images_set.astype(np.float64)

saver = D.Saver(test_path)
test_images_set, test_group_index = saver.load()
test_images_set = test_images_set.astype(np.float64)

In [ ]:
# bona fide example
g_l=1
# artefact
f_l=-1

In [ ]:
train_represetation = None
test_represetation = None
gamma = 0.00015
count = 0
for train_images, test_images in zip(np.transpose(train_images_set, (3,0,1,2)),\
                                     np.transpose(test_images_set, (3,0,1,2))) :
    # flatten the image
    train_images = np.array([img.flatten() for img in train_images])
    test_images = np.array([img.flatten() for img in test_images])
    print("After flattening, the shape of traing images: {}, the shape of testing images :{}".format(train_images.shape,\
                                                                                                  test_images.shape))

    # compute the DK feature from training set
    tmp_train_represetation, train_labels, pca = DK(train_images, train_group_index, is_traing=True, gamma=gamma)
    train_labels = np.array(train_labels)
    print(np.array(tmp_train_represetation).shape, np.array(train_labels).shape)

    # compute the DK feature from testing set
    tmp_test_represetation, test_labels = DK(test_images, test_group_index, is_traing=False, pca=pca, gamma=gamma)
    test_labels = np.array(test_labels)
    print(np.array(tmp_test_represetation).shape, np.array(test_labels).shape)

    if train_represetation is None :
        train_represetation = np.array(tmp_train_represetation)
        test_represetation = np.array(tmp_test_represetation)
    else :
        train_represetation = np.hstack((train_represetation, np.array(tmp_train_represetation)))
        test_represetation = np.hstack((test_represetation, np.array(tmp_test_represetation)))
    print("the shape of traing pre: {}, the shape of testing pre :{}".format(train_represetation.shape,\
                                                                             test_represetation.shape))

In [ ]:
train_DK_KM = Save_KM_label(train_represetation, train_represetation, train_labels, "linear")
test_DK_KM = Save_KM_label(test_represetation, train_represetation, test_labels, "linear")

In [ ]:
# for the training of SVM
svm_new = SVC(C=7, kernel="precomputed", class_weight='balanced')
print(train_DK_KM.shape, len(train_labels))
svm_new.fit(train_DK_KM, train_labels)
y_true, y_pred = train_labels, svm_new.predict(train_DK_KM)
print(classification_report(y_true, y_pred))
print(len(y_true), "   %d - genuine, %d - fake" % (len(np.where(y_true==1)[0]), len(np.where(y_true==-1)[0]) ))
print("accuracy: ", ((y_true==y_pred).sum())/len(y_true) )


# predict with trainging set
y_true, y_pred = test_labels, svm_new.predict(test_DK_KM)
# print(classification_report(y_true, y_pred))
# y_true, y_pred = list(y_true), list(y_pred)
print(len(y_true), "   %d - genuine, %d - fake" % (len(np.where(y_true==1)[0]), len(np.where(y_true==-1)[0]) ))
y_true = np.array(y_true)
print("accuracy: ", ((y_true==y_pred).sum())/len(y_true) )


# predict with testing set
y_true = np.array(y_true)
y_pred = np.array(y_pred)
tr_index = np.where(y_true==g_l)
fa_index = np.where(y_true==f_l)
FRR = len(np.where(y_pred[tr_index]==f_l)[0])/len(tr_index[0])
FAR = len(np.where(y_pred[fa_index]==g_l)[0])/len(fa_index[0])
HTER = (FAR+FRR)/2
print("FAR: %f   FRR: %f   HTER: %f" % (FAR, FRR, HTER))

In [ ]:
np.random.seed(3)
# get the deep features
train_deep, train_deep_labels, train_deep_group_keys = Deep(train_group_index, g_l=g_l, f_l=f_l, random=True)
test_deep, test_deep_labels, test_deep_group_keys = Deep(test_group_index, g_l=g_l, f_l=f_l, random=True)

In [ ]:
train_deep = np.array(train_deep)
test_deep = np.array(test_deep)
train_deep_labels = np.array(train_deep_labels)
test_deep_labels = np.array(test_deep_labels)
print(train_deep.shape, test_deep.shape)
print(train_deep_labels.shape, test_deep_labels.shape)

In [ ]:
tmp_train_deep = train_deep
tmp_test_deep = test_deep

In [ ]:
# get the average of deep features
tmp_train_deep = average_deepFea(train_deep, train_group_index)
tmp_test_deep = average_deepFea(test_deep, test_group_index)
print(tmp_train_deep.shape, tmp_test_deep.shape)
print(train_deep_labels.shape, test_deep_labels.shape)

In [ ]:
train_deep_KM = Save_KM_label(tmp_train_deep, tmp_train_deep, train_deep_labels, "rbf", gamma=5e-5)
test_deep_KM = Save_KM_label(tmp_test_deep, tmp_train_deep, test_deep_labels, "rbf", gamma=5e-5)

In [ ]:
# bona fide example
g_l=1
# artefact
f_l=-1

In [ ]:
# for the training of SVM
svm_new = SVC(C=1, kernel="precomputed", class_weight={1:10})
print(train_deep_KM.shape, len(train_deep_labels))
svm_new.fit(train_deep_KM, train_deep_labels)
y_true, y_pred = train_deep_labels, svm_new.predict(train_deep_KM)
print(classification_report(y_true, y_pred))
print(len(y_true), "   %d - genuine, %d - fake" % (len(np.where(y_true==1)[0]), len(np.where(y_true==-1)[0]) ))
print("accuracy: ", ((y_true==y_pred).sum())/len(y_true) )


# predict with trainging set
y_true, y_pred = test_deep_labels, svm_new.predict(test_deep_KM)
# print(classification_report(y_true, y_pred))
# y_true, y_pred = list(y_true), list(y_pred)
print(len(y_true), "   %d - genuine, %d - fake" % (len(np.where(y_true==1)[0]), len(np.where(y_true==-1)[0]) ))
y_true = np.array(y_true)
print("accuracy: ", ((y_true==y_pred).sum())/len(y_true) )


# predict with testing set
y_true = np.array(y_true)
y_pred = np.array(y_pred)
tr_index = np.where(y_true==g_l)
fa_index = np.where(y_true==f_l)
FRR = len(np.where(y_pred[tr_index]==f_l)[0])/len(tr_index[0])
FAR = len(np.where(y_pred[fa_index]==g_l)[0])/len(fa_index[0])
HTER = (FAR+FRR)/2
print("FAR: %f   FRR: %f   HTER: %f" % (FAR, FRR, HTER))

In [ ]:
DK_nor = np.trace(train_DK_KM) / (len(train_DK_KM) * len(train_DK_KM))
deep_nor = np.trace(train_deep_KM) / (len(train_deep_KM) * len(train_deep_KM))
print(DK_nor, deep_nor)

In [ ]:
tmp1 = 0.5
tmp2 = 1e-3
coef1 = tmp1/(tmp1+tmp2)
coef2 = tmp2/(tmp1+tmp2)
coef1 = 0.98
coef2 = 0.02
print(coef1, coef2)
train_KM = train_DK_KM/DK_nor*coef1 + train_deep_KM/deep_nor*coef2
test_KM = test_DK_KM/DK_nor*coef1 + test_deep_KM/deep_nor*coef2

In [ ]:
# for the training of SVM
svm_new = SVC(C=1, kernel="precomputed", class_weight={1:10})
print(train_KM.shape, len(train_labels))
svm_new.fit(train_KM, train_labels)
y_true, y_pred = train_labels, svm_new.predict(train_KM)
print(classification_report(y_true, y_pred))
print(len(y_true), "   %d - genuine, %d - fake" % (len(np.where(y_true==1)[0]), len(np.where(y_true==-1)[0]) ))
print("accuracy: ", ((y_true==y_pred).sum())/len(y_true) )


# predict with trainging set
y_true, y_pred = test_labels, svm_new.predict(test_KM)
# print(classification_report(y_true, y_pred))
# y_true, y_pred = list(y_true), list(y_pred)
print(len(y_true), "   %d - genuine, %d - fake" % (len(np.where(y_true==1)[0]), len(np.where(y_true==-1)[0]) ))
y_true = np.array(y_true)
print("accuracy: ", ((y_true==y_pred).sum())/len(y_true) )


# predict with testing set
y_true = np.array(y_true)
y_pred = np.array(y_pred)
tr_index = np.where(y_true==g_l)
fa_index = np.where(y_true==f_l)
FRR = len(np.where(y_pred[tr_index]==-1)[0])/len(tr_index[0])
FAR = len(np.where(y_pred[fa_index]==1)[0])/len(fa_index[0])
HTER = (FAR+FRR)/2
print("FAR: %f   FRR: %f   HTER: %f" % (FAR, FRR, HTER))